# Building Tools for Agents

In [1]:
import os
from SpinLLM import SpinChatModel

# models to try: llama3.2:latest, gemma3:1b, llama3:instruct, llama3-groq-tool-use:8b, 
# different models can lead to different results in tool use
# tool use requires careful finetuning of models and thus a lot of models fail to perform well for tool use
# try different models to see how certain models can fail or hallucinate when using tools
# the default model (llama3-groq-tool-use:8b) are fine tuned for tool use and thus provide a reasonable performance

llm = SpinChatModel(
    model_name="llama3-groq-tool-use:8b",
    encryption_path='/root/.spilli/SpiLLI.pem',
    temperature=0.8,
    max_tokens=512
)

/opt/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connecting using cus_id: cus_T42EpGqSccMdhc
[Socket] Connected
[Socket] Connected



## 1.  What Kind of Tools Can You Build?

| Tool Category | Typical Use‑Case | Example Implementation |
|---------------|------------------|------------------------|
| **Web Search** | Fetch latest info from the internet | `SerpAPI` wrapper |
| **Database Query** | Read/Write from SQL or NoSQL databases | `SQLDatabaseTool` |
| **File I/O** | Read local files (CSV, TXT, JSON) | `FileReadTool` |
| **API Calls** | Interact with external services (weather, finance) | `HTTPTool` |
| **Mathematics** | Perform calculations or symbolic math | `CalculatorTool` |
| **Data Processing** | Run Pandas transformations | `PandasTool` |
| **Summarisation** | Summarise long documents | `LLMSummarizer` |
| **Custom Business Logic** | Any domain‑specific operation | `BaseTool` subclass |

Below we’ll implement a handful of these tools from scratch using SpiLLI and LangChain.

---

## 2.  Building Custom Tools

All tools in LangChain inherit from `BaseTool`.  
The key components are:

1. **`name`** – short identifier used by the agent.
2. **`description`** – natural‑language explanation of what the tool does.
3. **`_run`** – the actual logic executed when the tool is called.

### 2.1  A Simple Calculator Tool


In [2]:

from langchain.tools import BaseTool

class CalculatorTool(BaseTool):
    name:str = "calculator"
    description:str = "Use this tool to perform basic arithmetic or advanced math operations. Input should be a mathematical expression."

    def _run(self, expression: str) -> str:
        try:
            # Warning: eval is dangerous; in production, use a math parser
            result = eval(expression, {"__builtins__": {}})
            return f"The result is: {result}"
        except Exception as e:
            return f"Error evaluating expression: {e}"

calculator_tool = CalculatorTool()




> **Security note** – Never expose `eval` to untrusted user input. In a real application, use a sandboxed math library (e.g., `sympy`, `mpmath`).

### 4.2  A File‑Reader Tool


In [3]:
import pathlib

class FileReadTool(BaseTool):
    name:str = "file_reader"
    description:str = "Read the contents of a file. Input should be a file path relative to the current working directory."

    def _run(self, path: str) -> str:
        try:
            file_path = pathlib.Path(path).resolve()
            if not file_path.exists():
                return f"File not found: {file_path}"
            content = file_path.read_text(encoding="utf-8")
            return f"Contents of {file_path}: {content}"
        except Exception as e:
            return f"Error reading file: {e}"

file_read_tool = FileReadTool()

In [4]:
file_read_tool.invoke('SpiLLI.txt')

'Contents of /app/Tutorials/Building_Agent_Tools/SpiLLI.txt: SpiLLI is used with LangChain to provide LLMs that can decide when and which tool to use and then generate a response using a chain-of-thought approach.'


### 2.3  An External API Call Tool (e.g., Weather API)


In [5]:
import requests

class WeatherTool(BaseTool):
    name:str = "weather_api"
    description:str = (
        "Fetch current weather information for a city. Input should be the city name, e.g., 'Paris'. "
        "You will need to register at https://openweathermap.org/api and set the environment variable "
        "OPENWEATHERMAP_API_KEY."
    )

    def _run(self, city: str) -> str:
        api_key = os.getenv("OPENWEATHERMAP_API_KEY")
        if not api_key:
            return "API key not configured. Set OPENWEATHERMAP_API_KEY environment variable."
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid={api_key}"
        try:
            resp = requests.get(url)
            data = resp.json()
            if resp.status_code != 200:
                return f"Error from API: {data.get('message', 'Unknown error')}"
            temp = data["main"]["temp"]
            desc = data["weather"][0]["description"]
            return f"The current temperature in {city} is {temp}°C with {desc}."
        except Exception as e:
            return f"Failed to fetch weather: {e}"

weather_tool = WeatherTool()


> **Tip** – Store your API keys securely (e.g., in a `.env` file or a secrets manager).



## 3.  Putting Tools in an AI Agent

We’ll now create an `AgentExecutor` that can choose among our tools based on a prompt.



Wrap tools into LangChain's Tool format


In [6]:
# from langchain.agents import initialize_agent, AgentType, Tool
from langchain_community.tools import Tool
from langchain_community.tools.file_management.read import ReadFileTool

tools = [
    Tool.from_function(
        func=calculator_tool._run,
        name=calculator_tool.name,
        description=calculator_tool.description
    ),
    Tool.from_function(
        func=file_read_tool._run,
        name=file_read_tool.name,
        description=file_read_tool.description
    ),
    Tool.from_function(
        func=weather_tool._run,
        name=weather_tool.name,
        description=weather_tool.description
    ),
]


# Initialize an agent that can decide when to use a tool


In [7]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
)


> **Tip** –  see what the internals look like for the agent.
>  you can change the agent type to different pre-built types, changing how the agents are prompted, which leads to different performance in terms of tool use success rate and halucination problems.
 You can try a few to get a sense of this vulnerability (see avilable pre-made agents at: https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html). 

> then in the exercises you will write custom agent types to see how you can optimize your prompts to get the best results for a given llm or fine tuning

---

## 4.  Running the Agent

Below are a few example queries to see the agent in action.

### 4.1  Simple Calculation


In [13]:
from langchain.messages import HumanMessage
agent.invoke({'messages':HumanMessage("What is 23 + 42?")})

{'messages': [HumanMessage(content='What is 23 + 42?', additional_kwargs={}, response_metadata={}, id='a91386fd-153e-46aa-b328-1c8988be7657'),
  AIMessage(content='', additional_kwargs={}, response_metadata={}, id='lc_run--20523617-88d7-46d7-a5ae-0d1a946aa7d5-0', tool_calls=[{'name': 'calculator', 'args': {'expression': '23 + 42'}, 'id': 'call_calculator_5497', 'type': 'tool_call'}]),
  ToolMessage(content='The result is: 65', name='calculator', id='a1c09044-8590-4c8a-8734-347be9303e93', tool_call_id='call_calculator_5497'),
  AIMessage(content='The result is: 65', additional_kwargs={}, response_metadata={}, id='lc_run--7fff7738-dfe3-42e1-a1a4-85d018400ec0-0')]}


*Expected output (something like):*  
```
Thinking...
Tool: calculator | Input: 23 + 42
Result: The result is: 65
...
Answer: The result is: 65.
```


### 4.2  Read a File


In [16]:
agent.invoke({"messages":HumanMessage("Please read the file ./SpiLLI.txt. and summarize its contents")})

{'messages': [HumanMessage(content='Please read the file ./SpiLLI.txt. and summarize its contents', additional_kwargs={}, response_metadata={}, id='fe6fc0aa-fa51-4b42-bc23-a2589a4f4e08'),
  AIMessage(content='', additional_kwargs={}, response_metadata={}, id='lc_run--0bdb24fb-8bd9-4e30-a5c8-1d87732475cc-0', tool_calls=[{'name': 'file_reader', 'args': {'file_path': './SpiLLI.txt'}, 'id': 'call_file_reader_9858', 'type': 'tool_call'}]),
  ToolMessage(content='Contents of /app/Tutorials/Building_Agent_Tools/SpiLLI.txt: SpiLLI is used with LangChain to provide LLMs that can decide when and which tool to use and then generate a response using a chain-of-thought approach.', name='file_reader', id='4c8be53f-7c4c-447f-8bd7-5a1d33d5d93f', tool_call_id='call_file_reader_9858'),
  AIMessage(content='The file ./SpiLLI.txt contains information about SpiLLI, which is used with LangChain to provide LLMs that can decide when and which tool to use and then generate a response using a chain-of-thought a

> **Observation:** You can note how the LLM generated the tool call to read the file and then summarizes the content it in the final answer. 


### 4.3  Weather Query


In [19]:
agent.invoke({"messages":HumanMessage("What's the current weather in New York?")})

{'messages': [HumanMessage(content="What's the current weather in New York?", additional_kwargs={}, response_metadata={}, id='b97e225e-ccec-4785-84c5-4bb88a532b42'),
  AIMessage(content='', additional_kwargs={}, response_metadata={}, id='lc_run--88e1f0c4-fbfd-42f9-be58-20f6951b81aa-0', tool_calls=[{'name': 'weather_api', 'args': {'city': 'New York'}, 'id': 'call_weather_api_2789', 'type': 'tool_call'}]),
  ToolMessage(content='API key not configured. Set OPENWEATHERMAP_API_KEY environment variable.', name='weather_api', id='ab95d1d1-cde6-4adc-8382-6345e0422a06', tool_call_id='call_weather_api_2789'),
  AIMessage(content="I'm sorry, but I can't fetch the current weather information for New York without the API key. Please set the OPENWEATHERMAP_API_KEY environment variable to proceed.", additional_kwargs={}, response_metadata={}, id='lc_run--a68fa486-4aa7-4df8-82da-6697f7c94ab6-0')]}


*Expected output:*  
```
{'messages': [HumanMessage(content="What's the current weather in New York?", additional_kwargs={}, response_metadata={}, id='b97e225e-ccec-4785-84c5-4bb88a532b42'),
  AIMessage(content='', additional_kwargs={}, response_metadata={}, id='lc_run--88e1f0c4-fbfd-42f9-be58-20f6951b81aa-0', tool_calls=[{'name': 'weather_api', 'args': {'city': 'New York'}, 'id': 'call_weather_api_2789', 'type': 'tool_call'}]),
  ToolMessage(content='API key not configured. Set OPENWEATHERMAP_API_KEY environment variable.', name='weather_api', id='ab95d1d1-cde6-4adc-8382-6345e0422a06', tool_call_id='call_weather_api_2789'),
  AIMessage(content="I'm sorry, but I can't fetch the current weather information for New York without the API key. Please set the OPENWEATHERMAP_API_KEY environment variable to proceed.", additional_kwargs={}, response_metadata={}, id='lc_run--a68fa486-4aa7-4df8-82da-6697f7c94ab6-0')]}
```

*Warning:* Watch for hullucinations in the generated response steps



## 5.  Extending the Toolkit



You can now create as many tools as you need. Some ideas:

| Domain | Possible Tool |
|--------|---------------|
| **Finance** | Stock price fetcher (Yahoo Finance, Alpha Vantage) |
| **NLP** | Text sentiment analyzer, entity extractor |
| **Knowledge Base** | Query a local knowledge graph or a vector store |
| **Automation** | Send emails, schedule calendar events |
| **Data Science** | Run a Pandas pipeline, train a lightweight model |

Just subclass `BaseTool`, implement `_run`, and add it to the `tools` list.

---

## 6.  Recap

1. **Install** LangChain and dependencies.  
2. **Set up** an LLM using SpiLLI.  
3. **Implement** tools by inheriting `BaseTool`.  
4. **Wrap** each tool in LangChain’s `Tool` object.  
5. **Create** an `Agent` that can reason and choose tools.  
6. **Run** the agent with natural‑language prompts.

Happy hacking! 🚀

## 7. Further Study: Writing a Custom Agent Executor

### Exercise: Look at how custom agent graphs can be written using LangGraph and use your custom agent flow instead of the above default agent if you are feeling ambitious. You'll learn a lot beyond just using the above default agent framework.